<a href="https://colab.research.google.com/github/manglesh001/DL-assigment1/blob/main/DL_Ass1_Q3_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from sklearn.preprocessing import OneHotEncoder
import wandb

In [25]:
# Load the Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [26]:
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0

In [27]:
# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

In [28]:
# Activation functions and derivatives
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [29]:
#tanh activation and its derivative
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

In [30]:
#relu activation and its derivative

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [31]:
# Weight Initialization
def initialize_weights(layers, method="random"):
    weights = []
    biases = []
    for i in range(len(layers) - 1):
        if method == "xavier":
            weights.append(np.random.randn(layers[i], layers[i+1]) * np.sqrt(1 / layers[i]))
        else:  # random
            weights.append(np.random.randn(layers[i], layers[i+1]) * 0.01)
        biases.append(np.zeros((1, layers[i+1])))
    return weights, biases

In [32]:
# Forward Propagation
def forward_propagation(X, weights, biases, activation):
    activations = [X]
    zs = []

    for i in range(len(weights)):
        z = np.dot(activations[-1], weights[i]) + biases[i]
        zs.append(z)

        if activation[i] == "sigmoid":
            activations.append(sigmoid(z))
        elif activation[i] == "relu":
            activations.append(relu(z))
        elif activation[i] == "tanh":
            activations.append(tanh(z))

    return activations, zs

In [33]:
# Backpropagation
def backpropagation(y, activations, zs, weights, activation):
    gradients_w = [None] * len(weights)
    gradients_b = [None] * len(weights)

    # Output layer error
    error = activations[-1] - y

    for i in reversed(range(len(weights))):
        if activation[i] == "sigmoid":
            delta = error * sigmoid_derivative(activations[i+1])
        elif activation[i] == "relu":
            delta = error * relu_derivative(activations[i+1])
        elif activation[i] == "tanh":
            delta = error * tanh_derivative(activations[i+1])

        gradients_w[i] = np.dot(activations[i].T, delta)
        gradients_b[i] = np.sum(delta, axis=0, keepdims=True)

        error = np.dot(delta, weights[i].T)

    return gradients_w, gradients_b

In [34]:
#Optimizer Momentum
def momentum(weights, biases, gradients_w, gradients_b, lr, velocity, beta=0.9):
    for i in range(len(weights)):
        velocity[i] = beta * velocity[i] + (1 - beta) * gradients_w[i]
        weights[i] -= lr * velocity[i]
        biases[i] -= lr * gradients_b[i]
    return weights, biases, velocity

In [35]:
#Adam Optimizer
def adam(weights, biases, gradients_w, gradients_b, lr, m_w, v_w, m_b, v_b, beta1=0.9, beta2=0.999, epsilon=1e-8, t=1):
    for i in range(len(weights)):
        # Update for weights
        m_w[i] = beta1 * m_w[i] + (1 - beta1) * gradients_w[i]
        v_w[i] = beta2 * v_w[i] + (1 - beta2) * (gradients_w[i] ** 2)
        m_hat_w = m_w[i] / (1 - beta1 ** t)
        v_hat_w = v_w[i] / (1 - beta2 ** t)
        weights[i] -= lr * m_hat_w / (np.sqrt(v_hat_w) + epsilon)

        # Update for biases
        m_b[i] = beta1 * m_b[i] + (1 - beta1) * gradients_b[i]
        v_b[i] = beta2 * v_b[i] + (1 - beta2) * (gradients_b[i] ** 2)
        m_hat_b = m_b[i] / (1 - beta1 ** t)
        v_hat_b = v_b[i] / (1 - beta2 ** t)
        biases[i] -= lr * m_hat_b / (np.sqrt(v_hat_b) + epsilon)

    return weights, biases, m_w, v_w, m_b, v_b

In [36]:
#optimizer Nesterov
def nesterov(weights, biases, gradients_w, gradients_b, lr, velocity, beta=0.9):
    for i in range(len(weights)):
        temp_velocity = beta * velocity[i]
        weights[i] -= lr * (beta * temp_velocity + (1 - beta) * gradients_w[i])
        biases[i] -= lr * gradients_b[i]
        velocity[i] = temp_velocity + (1 - beta) * gradients_w[i]
    return weights, biases, velocity

In [37]:
#RMSProp Otimizer
def rmsprop(weights, biases, gradients_w, gradients_b, lr, cache_w, cache_b, beta=0.99, epsilon=1e-8):
    for i in range(len(weights)):
        # Update cache for weights
        cache_w[i] = beta * cache_w[i] + (1 - beta) * (gradients_w[i] ** 2)
        weights[i] -= lr * gradients_w[i] / (np.sqrt(cache_w[i]) + epsilon)

        # Update cache for biases
        cache_b[i] = beta * cache_b[i] + (1 - beta) * (gradients_b[i] ** 2)
        biases[i] -= lr * gradients_b[i] / (np.sqrt(cache_b[i]) + epsilon)

    return weights, biases, cache_w, cache_b


In [18]:
#Nadam Optimizer

# def nadam(weights, biases, gradients_w, gradients_b, lr, m, v, beta1=0.9, beta2=0.999, epsilon=1e-8, t=1):
#     for i in range(len(weights)):
#         m[i] = beta1 * m[i] + (1 - beta1) * gradients_w[i]
#         v[i] = beta2 * v[i] + (1 - beta2) * (gradients_w[i] ** 2)
#         m_hat = (beta1 * m[i] + (1 - beta1) * gradients_w[i]) / (1 - beta1 ** t)
#         v_hat = v[i] / (1 - beta2 ** t)
#         weights[i] -= lr * m_hat / (np.sqrt(v_hat) + epsilon)
#         biases[i] -= lr * gradients_b[i] / (np.sqrt(v_hat) + epsilon)
#     return weights, biases, m, v

In [38]:
# Optimizers SGD
def sgd(weights, biases, gradients_w, gradients_b, lr):
    for i in range(len(weights)):
        weights[i] -= lr * gradients_w[i]
        biases[i] -= lr * gradients_b[i]
    return weights, biases

In [40]:
#Load Fashion-MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], -1) / 255.0
X_test = X_test.reshape(X_test.shape[0], -1) / 255.0

y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

In [42]:

def train_network(X_train, y_train, X_val, y_val, config):
    np.random.seed(42)
    layers = [X_train.shape[1]] + [config['hidden_size']] * config['hidden_layers'] + [10]
    activation = [config['activation']] * config['hidden_layers'] + ['sigmoid']

    weights, biases = initialize_weights(layers, config['weight_init'])
    optimizer = config['optimizer']

    velocity = [np.zeros_like(w) for w in weights]
    cache_w = [np.zeros_like(w) for w in weights]
    cache_b = [np.zeros_like(b) for b in biases]
    m_w = [np.zeros_like(w) for w in weights]  # For weights
    v_w = [np.zeros_like(w) for w in weights]  # For weights
    m_b = [np.zeros_like(b) for b in biases]   # For biases
    v_b = [np.zeros_like(b) for b in biases]   # For biases

    batch_size = config['batch_size']
    epochs = config['epochs']
    lr = config['learning_rate']

    for epoch in range(epochs):
        indices = np.random.permutation(X_train.shape[0])
        X_train_shuffled, y_train_shuffled = X_train[indices], y_train[indices]

        train_loss = 0
        train_correct = 0
        train_total = 0

        for i in range(0, X_train_shuffled.shape[0], batch_size):
            X_batch = X_train_shuffled[i:i+batch_size]
            y_batch = y_train_shuffled[i:i+batch_size]

            # Forward propagation
            activations, zs = forward_propagation(X_batch, weights, biases, activation)

            # Calculate training loss (cross-entropy loss)
            output = activations[-1]

            train_loss += -np.sum(y_batch * np.log(output + 1e-8)) / len(y_batch)
            #calculate mean squared error
           # train_loss += np.mean(np.square(activations[-1] - y_batch))

            # Calculate training accuracy
            train_preds = np.argmax(output, axis=1)
            train_true = np.argmax(y_batch, axis=1)
            train_correct += np.sum(train_preds == train_true)
            train_total += len(y_batch)

            # Backpropagation
            gradients_w, gradients_b = backpropagation(y_batch, activations, zs, weights, activation)

            # Update weights and biases based on optimizer
            if optimizer == 'sgd':
                weights, biases = sgd(weights, biases, gradients_w, gradients_b, lr)
            elif optimizer == 'momentum':
                weights, biases, velocity = momentum(weights, biases, gradients_w, gradients_b, lr, velocity)
            elif optimizer == 'nesterov':
                weights, biases, velocity = nesterov(weights, biases, gradients_w, gradients_b, lr, velocity)
            elif optimizer == 'rmsprop':
                weights, biases, cache_w, cache_b = rmsprop(weights, biases, gradients_w, gradients_b, lr, cache_w, cache_b)
            elif optimizer == 'adam':
                weights, biases, m_w, v_w, m_b, v_b = adam(weights, biases, gradients_w, gradients_b, lr, m_w, v_w, m_b, v_b, t=epoch+1)
            # elif optimizer == 'nadam':
            #     pass

        # Calculate average training loss and accuracy for the epoch
        train_loss /= (X_train_shuffled.shape[0] // batch_size)
        train_accuracy = train_correct / train_total

        # Validate model
        val_activations, _ = forward_propagation(X_val, weights, biases, activation)
        val_loss = -np.sum(y_val * np.log(val_activations[-1] + 1e-8)) / len(y_val)
        val_accuracy = np.mean(np.argmax(val_activations[-1], axis=1) == np.argmax(y_val, axis=1))

        # Log metrics to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_accuracy,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy
        })

        # Print metrics
        print(f"Epoch {epoch + 1}/{epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    return

In [43]:
# Split training data into train and validation
X_train, X_val = X_train[:54000], X_train[54000:]
y_train, y_val = y_train[:54000], y_train[54000:]

In [44]:
# Hyperparameter tuning with wandb
sweep_config = {
    'method': 'random',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'epochs': {'values': [5, 10]},
        'hidden_layers': {'values': [3, 4, 5]},
        'hidden_size': {'values': [32, 64, 128]},
        'weight_decay': {'values': [0, 0.0005, 0.5]},
        'learning_rate': {'values': [1e-3, 1e-4]},
        'optimizer': {'values': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']},
        'batch_size': {'values': [16, 32, 64]},
        'weight_init': {'values': ['random', 'xavier']},
        'activation': {'values': ['sigmoid', 'tanh', 'relu']}
    }
}

In [45]:
#wandb initize and sweep rename  call train
def train():
    wandb.init()

    sweep_name = (
        f"hl_{wandb.config.hidden_layers}_"
        f"hs_{wandb.config.hidden_size}_"
        f"bs_{wandb.config.batch_size}_"
        f"ac_{wandb.config.activation}_"
        f"opt_{wandb.config.optimizer}_"
        f"lr_{wandb.config.learning_rate}_"
        f"init_{wandb.config.weight_init}"
    )
    wandb.run.name = sweep_name

    config = wandb.config
    train_network(X_train, y_train, X_val, y_val, config)

In [46]:
sweep_id = wandb.sweep(sweep_config, project="fashion-mnist")
wandb.agent(sweep_id, function=train, count=20)
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: wk1e0itn
Sweep URL: https://wandb.ai/mangleshpatidar2233-iit-madras-alumni-association/fashion-mnist/sweeps/wk1e0itn


wandb: Agent Starting Run: ilbxb9a6 with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_layers: 5
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.5
wandb: 	weight_init: xavier
wandb: Currently logged in as: mangleshpatidar2233 (mangleshpatidar2233-iit-madras-alumni-association) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Epoch 1/5, Train Loss: 0.6587, Train Accuracy: 0.1062, Val Loss: 0.6575, Val Accuracy: 0.1103
Epoch 2/5, Train Loss: 0.6587, Train Accuracy: 0.1062, Val Loss: 0.6575, Val Accuracy: 0.1103
Epoch 3/5, Train Loss: 0.6587, Train Accuracy: 0.1062, Val Loss: 0.6575, Val Accuracy: 0.1103
Epoch 4/5, Train Loss: 0.6587, Train Accuracy: 0.1062, Val Loss: 0.6575, Val Accuracy: 0.1103
Epoch 5/5, Train Loss: 0.6587, Train Accuracy: 0.1062, Val Loss: 0.6575, Val Accuracy: 0.1103


epoch,▁▃▅▆█
train_accuracy,▁▁▁▁▁
train_loss,▂▂▁▆█
val_accuracy,▁▁▁▁▁
val_loss,▁▁▁▁▁
epoch,5
train_accuracy,0.10622
train_loss,0.65874
val_accuracy,0.11033
val_loss,0.65752


wandb: Agent Starting Run: 3vu6pza9 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_layers: 5
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: nesterov
wandb: 	weight_decay: 0
wandb: 	weight_init: random


Epoch 1/5, Train Loss: 1.6266, Train Accuracy: 0.0986, Val Loss: 2.0157, Val Accuracy: 0.0985
Epoch 2/5, Train Loss: 2.1289, Train Accuracy: 0.0987, Val Loss: 2.2037, Val Accuracy: 0.0942
Epoch 3/5, Train Loss: 2.2389, Train Accuracy: 0.0987, Val Loss: 2.2643, Val Accuracy: 0.0942
Epoch 4/5, Train Loss: 2.2774, Train Accuracy: 0.1000, Val Loss: 2.2873, Val Accuracy: 0.0925
Epoch 5/5, Train Loss: 2.2924, Train Accuracy: 0.1008, Val Loss: 2.2965, Val Accuracy: 0.0925


epoch,▁▃▅▆█
train_accuracy,▁▁▁▅█
train_loss,▁▆▇██
val_accuracy,█▃▃▁▁
val_loss,▁▆▇██
epoch,5
train_accuracy,0.10081
train_loss,2.2924
val_accuracy,0.0925
val_loss,2.29646
